# MiniCPM-2B 参数高效微调（LoRA）A100 80G 单卡示例

显存更小的显卡可用 batch size 和 grad_accum 间时间换空间

本 notebook 是一个使用 `AdvertiseGen` 数据集对 MiniCPM-2B 进行 LoRA 微调，使其具备专业的广告生成能力的代码示例。

## 最低硬件需求
- 显存：12GB
- 显卡架构：安培架构（推荐）
- 内存：16GB

## 1. 准备数据集

将数据集转换为更通用的格式


In [7]:
# 转换为 ChatML 格式
import os
import shutil
import json

input_dir = "data/AdvertiseGen"
output_dir = "data/AdvertiseGenChatML"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

for fn in ["train.json", "dev.json"]:
    data_out_list = []
    with open(os.path.join(input_dir, fn), "r") as f, open(os.path.join(output_dir, fn), "w") as fo:
        for line in f:
            if len(line.strip()) > 0:
                data = json.loads(line)
                data_out = {
                    "messages": [
                        {
                            "role": "user",
                            "content": data["content"],
                        },
                        {
                            "role": "assistant",
                            "content": data["summary"],
                        },
                    ]
                }
                data_out_list.append(data_out)
        json.dump(data_out_list, fo, ensure_ascii=False, indent=4)


## 2. 使用 LoRA 进行微调

命令行一键运行

In [8]:
!bash lora_finetune.sh

20240515142253
[2024-05-15 14:22:55,122] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-15 14:22:56,390] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-05-15 14:22:56,401] [INFO] [runner.py:568:main] cmd = /home/mnt/wyx/anaconda3/envs/finetune/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None finetune.py --model_name_or_path /home/mnt/wyx/src/Finetune-MiniCPM/MiniCPM-2B-sft-bf16 --output_dir output/AdvertiseGenLoRA/20240515142253/ --train_data_path data/AdvertiseGenChatML/train.json --eval_data_path data/AdvertiseGenChatML/dev.json --learning_rate 5e-5 --per_device_train_batch_size 2 --per_device_eval_batch_size 2 --model_max_length 384 --bf16 --use_lora --gradient_accumulation_steps 1 --warmup_steps 100 --max_steps 3000 --weight_decay 0.01 --evaluatio

## 推理验证

In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
path = "output/AdvertiseGenLoRA/20240315224356/checkpoint-3000"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.bfloat16, device_map="cuda", trust_remote_code=True
)

OSError: /home/mnt/wyx/src/Finetune-MiniCPM/finetune/output/AdvertiseGenLoRA/20240515145527 does not appear to have a file named config.json. Checkout 'https://huggingface.co//home/mnt/wyx/src/Finetune-MiniCPM/finetune/output/AdvertiseGenLoRA/20240515145527/tree/None' for available files.

In [ ]:
res, history = model.chat(tokenizer, query="<用户>类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞<AI>", max_length=80, top_p=0.5)
res, history

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer



In [3]:
# 假设你已经加载了模型和tokenizer
model_path = "output/AdvertiseGenLoRA/20240515152537/"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [4]:
res, history = model.chat(tokenizer, query="<用户>类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞<AI>", max_length=80, top_p=0.5)
res, history

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


('这款牛仔外套，采用白色牛仔布面料，简约的白色，更显干净利落。破洞的装饰，增加了几分时尚感，同时又不失个性。简约的',
 [{'role': 'user',
   'content': '<用户>类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞<AI>'},
  {'role': 'assistant',
   'content': '这款牛仔外套，采用白色牛仔布面料，简约的白色，更显干净利落。破洞的装饰，增加了几分时尚感，同时又不失个性。简约的'}])